In [2]:
#import modules
!pip install nltk
from google.colab import files
import pandas as pd
import io
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize,RegexpTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics #accuracy calculation
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#functions for loading data

def mount_drive():
  drive.mount("/content/drive")

def load_subreddit_data(filename="reddit_train.csv"):
  uploaded = files.upload()
  return pd.read_csv(io.BytesIO(uploaded[filename]), encoding='latin1')

In [5]:
#load data
X = load_subreddit_data()

Saving reddit_train.csv to reddit_train.csv


In [6]:
X_new = X.copy()

In [9]:
X = X_new

In [8]:
#Data Processing from Cheng Siong
#Convert comments to tokens
X['BODY'] = X['BODY'].fillna("").map(word_tokenize)
X['Frequency'] = X['BODY'].map(FreqDist)
#stop_words = stopwords.words("english")

def filterSentence(sentence):
    filteredSentence = []
    for w in sentence:
      if w not in stop_words:
        filteredSentence.append(w)
    return filteredSentence

X['Filtered'] = X['BODY']#.map(filterSentence)
X

KeyboardInterrupt: ignored

In [10]:
# Kok's data preprocessing
import re
corpus = []
original_data = X['BODY']
for i in range(len(X)):
  comment = re.sub('[^a-zA-Z]',' ', X['BODY'][i])
  comment = comment.lower().split()
  ps = PorterStemmer()
  comment = [ps.stem(word) for word in comment if not word in set(stopwords.words('english'))]
  commment = ' '.join(comment)
  corpus.append(comment)
  corpus

In [ ]:
# Finalize X and y
from sklearn.feature_extraction.text import CountVectorizer
y = X["REMOVED"]
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(X["BODY"])

In [ ]:
# split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
#train model

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=400, criterion='entropy', random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
#visualize results
from sklearn.metrics import confusion_matrix
cm2 = confusion_matrix(y_test, y_pred)
cm2

array([[2665,  260],
       [ 904,  439]])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 
print(classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

           0       0.75      0.91      0.82      2925
           1       0.63      0.33      0.43      1343

    accuracy                           0.73      4268
   macro avg       0.69      0.62      0.63      4268
weighted avg       0.71      0.73      0.70      4268



In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7272727272727273


In [ ]:
#save training results
import numpy as np
y_test = np.asarray(y_test)
misclass = np.where(y_test != y_pred)[0]
false_positives = np.where((y_test == 1) &  (y_pred==0))[0]
false_negatives = np.where((y_test == 0) &  (y_pred==1))[0]
for i in range(10):
  print(original_data[false_negatives[i]])

This is indicative of a typical power law, and is found in any number of areas.  I'm actually a bit surprised that it is only half.
Well, I'm going to say that a risk we're willing to take
Could it be that drinkers are purposefully going to surrounding areas to buy soda thus deflating sales in one place and inflating them in another?
Can I get a 3d printed spine some day? My spinal stenosis and sciatica would hate it.
At scale, this technology is what we need to get the carbon out of the atmosphere and back into the ground where it belongs.

That sounds ridiculous but it's true- everything on our planet evolved in an environment where that carbon was buried in the ground and inaccessible. Not in the air. 
Many folks seem to be rather flippant about this, but there is more value in it than they may realize.

As someone that has endured three lower back injuries I am keenly aware of just how early in the process of injury and pain management that we are funneled into PT. What too many PT

In [ ]:
print(false_negatives)

[   2   34   37   43   49   59   69   95   99  125  132  134  179  253
  254  284  287  301  320  405  450  452  486  575  579  583  584  585
  606  648  685  692  698  721  729  737  762  775  779  791  803  820
  830  845  859  866  870  885  910  921  938  981  984 1017 1018 1025
 1038 1044 1060 1090 1116 1169 1195 1199 1216 1251 1270 1295 1322 1324
 1350 1352 1371 1374 1394 1408 1423 1432 1476 1481 1483 1518 1528 1535
 1557 1561 1581 1587 1605 1611 1613 1649 1652 1656 1705 1721 1735 1737
 1739 1745 1754 1767 1784 1820 1839 1846 1864 1876 1884 1900 1907 1913
 1932 1946 1967 1970 1972 1975 2039 2040 2056 2067 2155 2160 2162 2182
 2194 2195 2209 2213 2218 2219 2225 2232 2237 2238 2241 2242 2246 2306
 2321 2347 2351 2358 2365 2380 2383 2404 2449 2459 2470 2481 2494 2512
 2576 2582 2589 2595 2619 2620 2643 2684 2703 2722 2748 2786 2790 2802
 2811 2814 2818 2825 2848 2856 2858 2891 2907 2921 2984 2995 3005 3013
 3014 3020 3031 3046 3063 3075 3077 3104 3114 3151 3167 3182 3203 3205
 3218 